In [9]:
import mlflow

In [5]:
from train import run
import os
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [14]:
def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def run(data_path):
    mlflow.sklearn.autolog()
    with mlflow.start_run():
        X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
        X_valid, y_valid = load_pickle(os.path.join(data_path, "valid.pkl"))

        rf = RandomForestRegressor(max_depth=10, random_state=0)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_valid)

        rmse = mean_squared_error(y_valid, y_pred, squared=False)


In [15]:
%time run('/Users/m.shark/Documents/mlops/mlops-zoomcamp/02-experiment-tracking/homework/output/')

2022/05/29 23:27:15 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/m.shark/anaconda3/envs/venv_mlops/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


CPU times: user 8.76 s, sys: 247 ms, total: 9.01 s
Wall time: 14.8 s


In [13]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("exp1")

2022/05/29 23:24:29 INFO mlflow.tracking.fluent: Experiment with name 'exp1' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='exp1', tags={}>

In [16]:
import argparse
import os
import pickle

import mlflow
import numpy as np
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("random-forest-hyperopt")


def load_pickle(filename):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def run(data_path, num_trials):


    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_valid, y_valid = load_pickle(os.path.join(data_path, "valid.pkl"))

    def objective(params):
        with mlflow.start_run():
            mlflow.log_params(params)
            
            rf = RandomForestRegressor(**params)
            rf.fit(X_train, y_train)
            y_pred = rf.predict(X_valid)
            rmse = mean_squared_error(y_valid, y_pred, squared=False)
            
            mlflow.log_metric("rmse", rmse)
            return {'loss': rmse, 'status': STATUS_OK}

    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
        'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
        'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
        'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
        'random_state': 42
    }

    rstate = np.random.default_rng(42)  # for reproducible results
    fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=num_trials,
        trials=Trials(),
        rstate=rstate
    )

In [18]:
run(data_path = "/Users/m.shark/Documents/mlops/mlops-zoomcamp/02-experiment-tracking/homework/output/", num_trials = 50)

  0%|                                    | 0/50 [00:00<?, ?trial/s, best loss=?]

2022/05/29 23:44:37 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 21d6b25b2bf743678e91cc228d1920e9. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Changing param values is not allowed. Params were already logged=\'[{\'key\': \'bootstrap\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'ccp_alpha\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'criterion\', \'old_value\': None, \'new_value\': \'squared_error\'}, {\'key\': \'max_features\', \'old_value\': None, \'new_value\': \'1.0\'}, {\'key\': \'max_leaf_nodes\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'max_samples\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'min_impurity_decrease\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'min_weight_fractio

  2%|▏          | 1/50 [00:06<05:18,  6.51s/trial, best loss: 6.658956269343007]

2022/05/29 23:44:38 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID ef90017fb3ce40e4b3a4e79b81dafb24. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Changing param values is not allowed. Params were already logged=\'[{\'key\': \'bootstrap\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'ccp_alpha\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'criterion\', \'old_value\': None, \'new_value\': \'squared_error\'}, {\'key\': \'max_features\', \'old_value\': None, \'new_value\': \'1.0\'}, {\'key\': \'max_leaf_nodes\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'max_samples\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'min_impurity_decrease\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'min_weight_fractio

  4%|▍          | 2/50 [00:07<02:40,  3.34s/trial, best loss: 6.658956269343007]

...

 96%|████████▋| 48/50 [03:54<00:07,  3.53s/trial, best loss: 6.6284257482044735]

2022/05/29 23:48:29 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 60dce880dce24cbabfaa8a79b310bc7d. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Changing param values is not allowed. Params were already logged=\'[{\'key\': \'bootstrap\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'ccp_alpha\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'criterion\', \'old_value\': None, \'new_value\': \'squared_error\'}, {\'key\': \'max_features\', \'old_value\': None, \'new_value\': \'1.0\'}, {\'key\': \'max_leaf_nodes\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'max_samples\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'min_impurity_decrease\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'min_weight_fractio

 98%|████████▊| 49/50 [03:58<00:03,  3.65s/trial, best loss: 6.6284257482044735]

2022/05/29 23:48:34 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID f0811472123847ada33509466175052f. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Changing param values is not allowed. Params were already logged=\'[{\'key\': \'bootstrap\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'ccp_alpha\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'criterion\', \'old_value\': None, \'new_value\': \'squared_error\'}, {\'key\': \'max_features\', \'old_value\': None, \'new_value\': \'1.0\'}, {\'key\': \'max_leaf_nodes\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'max_samples\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'min_impurity_decrease\', \'old_value\': None, \'new_value\': \'0.0\'}, {\'key\': \'min_weight_fractio

100%|█████████| 50/50 [04:03<00:00,  4.87s/trial, best loss: 6.6284257482044735]
